In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
from feather import read_dataframe
%matplotlib inline

In [2]:
complete = read_dataframe("user_book_ratings.feather",).pivot_table("Book-Rating",
                                                                    index="User-ID",
                                                                    columns="Book-Title")

In [3]:
complete.head(10)

Book-Title,Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",""" Lamb to the Slaughter and Other Stories (Penguin 60s S.)","""A"" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)","""O"" Is for Outlaw","""Surely You're Joking, Mr. Feynman!"": Adventures of a Curious Character","""The Happy Prince"" and Other Stories (Penguin Popular Classics)",'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,...,Zodiac: The Eco-Thriller,"Zoey Phillips (Girlfriends) (Harlequin Superromance, No. 1020)",Zombies of the Gene Pool,Zoya,Zoya's Story: An Afghan Woman's Struggle for Freedom,ZwÃ?Â¶lf.,e,iI Paradiso Degli Orchi,one hundred years of solitude,stardust
User-ID,,,,,,,,,,,,,,,,,,,,,
User_100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User_100098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
complete.memory_usage().sum() / 1024 ** 3

1.2238024920225143

In [5]:
def get_user(i, df):
    """Get row i; should be the same dtype as RowIndex"""
    return df.loc[i].values

def get_item(j, df):
    """Get row j from the transponsed matrix, aka item j"""
    return df.loc[:,j].values

def compute_evaluation_metric(y_true, y_pred, metric="MSE"):
    """Given two vectors computes the selected metric."""
    
    # This works only on the common elements
    # missing values are ignored
    diff = np.subtract(y_true, y_pred)
    
    # Count NaNs in the difference, check for NaNs and 
    # invert that, finally count non-zeros 
    # https://stackoverflow.com/a/21778195/6655150
    N = np.count_nonzero(~np.isnan(diff))
    
    if metric == "MSE":
        distance = (1 / N) * np.nansum( np.power(diff, 2) )
    elif metric == "MAE":
        distance = (1 / N) * np.nansum( np.abs(diff) )
    elif metric == "RMSE":
        distance = np.sqrt((1 / N) * np.nansum( np.power(diff, 2) ))
    elif metric == "R2":
        y_mean = np.nanmean(y_true)
        diffs1 = np.nansum(np.square(y_true - y_mean))
        diffs2 = np.nansum(np.square(y_pred - y_true))
        distance = 1 - diffs2 / diffs1
        
    return distance


def cos_sim(user, others):
    """
        This is nothing more than the implementation of cosine similarity
        in a way to handle (ignore) missing values. It is about 2-3x faster
        than replacing NaNs with zeros and then using scipy's or sklearn's
        functions. It assumes the input vectors are `atleast_2D`.
    """
    
    # add some numbers for numeric stability
    sum_sq = np.nansum(np.multiply(user, others), axis=1)

    magnitudes = np.multiply(np.sqrt(np.nansum(np.square(user))),
                             np.sqrt(np.nansum(np.square(others), axis=1))) 
    
    # calculate cosing similarity and add 
    # a small amount to avoid division by 0
    return np.divide(sum_sq+1e-7, magnitudes+1e-3).reshape((user.shape[0], -1))

def pearson_sim(user, others):
    """
        Similarly to the calculation of cosine similarity, this custom approach
        is faster (~1.5x) than filling the whole matrix (this time with the row
        mean) and then using `scipy.stats.stats.pearsonr` with a list comprehension
        iterating over all the rows.
    """
    
    user_mean = np.nanmean(user)
    others_means = np.nanmean(others, axis=1, keepdims=True)

    diff1 = np.subtract( user, user_mean ) 
    diff2 = np.subtract( others, others_means )

    var1 = np.sqrt( np.nansum( np.power(diff1, 2) ) )
    var2 = np.sqrt( np.nansum( np.power(diff2, 2), axis=1, 
                               keepdims=True) )

    num = np.nansum( np.multiply( diff1, diff2 ), axis=1,
                     keepdims=True )
    denom = np.multiply( var1, var2 )
    
    return np.divide(num, (denom + 1e-3)).reshape((user.shape[0], -1))

def similirity_evaluation(user, others, method="pearson"):
    """
        Given two vectors, or a vector and list of vectors, computes
        the selected similarity metric(s).
    """
    
    # make sure they are in the correct shape
    user = np.atleast_2d(user)
    others = np.atleast_2d(others)

    if method == "pearson":
        return pearson_sim(user, others)
    
    elif method == "cosine":
        return cos_sim(user, others)
    
def predict_user_ratings(user, others):
    
    # make sure the shape is correct
    user = np.atleast_2d(user)
    others = np.atleast_2d(others)

    # use numpy to calculate the mean ignoring NaNs
    user_mean = np.nanmean(user)
    others_means =  np.nanmean(others, axis=1, keepdims=True)
    
    diff2 = np.subtract( others, others_means )
    
    similarities = similirity_evaluation(user, others, "cosine")
    
    num = np.nansum( np.multiply(similarities.T, diff2), axis=0 )
    denom = np.nansum(similarities)
            
    return np.add(user_mean, np.divide(num, denom + 1e-4))

In [6]:
def find_similar_users(user, df, metric="pearson"):
    """
        Given a user and a dataframe (optionally a similarity metric)
        this function finds users
    """
    
    sims = similirity_evaluation(user, df, metric)
    results = pd.DataFrame([df.index, sims.ravel()],
                           index=["Users", "Similarity scores"]).T
    return results.sort_values("Similarity scores", ascending=False).reset_index(drop=True)

def find_similar_items(item, df, metric="cosine"):
    sims = similirity_evaluation(item, df.T, metric)
    results = pd.DataFrame([df.columns, sims.ravel()],
                           index=["Items", "Similarity scores"]).T
    return results.sort_values("Similarity scores", ascending=False).reset_index(drop=True)

In [7]:
i1 = get_user("User_8", complete)
i2 = get_user("User_11400", complete)
i3 = get_user(["User_11400", "User_67544"], complete)
i4 = get_user("User_67544", complete)

j1 = get_item(' Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback))', complete)
j2 = get_item('"A" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)', complete)
j3 = get_item(['Zodiac: The Eco-Thriller', 
               'Zoey Phillips (Girlfriends) (Harlequin Superromance, No. 1020)',], complete)
j4 = get_item(['\'Salem\'s Lot', '"O" Is for Outlaw', 
              ' Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback))'], complete)
j5 = get_item('Harry Potter and the Chamber of Secrets (Book 2)',complete)

(compute_evaluation_metric(i1, i2, "RMSE"), compute_evaluation_metric(i1, i2, "MAE"),
 compute_evaluation_metric(i1, i2, "MSE"))

(5.0, 5.0, 25.0)

In [8]:
sims = similirity_evaluation(get_item('Harry Potter and the Chamber of Secrets (Book 2)',complete),
                      complete.T, "cosine")
sims

array([[4.41018016e-11, 4.14264303e-11, 9.01698410e-11, ...,
        2.90427559e-11, 4.05922560e-11, 2.91007505e-02]])

In [9]:
user_itempreds = predict_user_ratings(i2, complete)
sims = similirity_evaluation(i2, complete)

In [10]:
test = pd.DataFrame(np.vstack((user_itempreds, i2, complete.columns))).T
test

,0,1,2
0,3.05291,NaN,Murder of a Sleeping Beauty (Scumble River My...
1,3.05475,NaN,"Q-Space (Star Trek The Next Generation, Book 47)"
2,3.04952,NaN,"Q-Zone (Star Trek The Next Generation, Book 48)"
3,3.05561,NaN,""" Lamb to the Slaughter and Other Stories (Pen..."
4,3.05374,NaN,"""A"" is for Alibi : A Kinsey Millhone Mystery (..."
5,3.03652,NaN,"""O"" Is for Outlaw"
6,3.05637,NaN,"""Surely You're Joking, Mr. Feynman!"": Adventur..."
7,3.04657,NaN,"""The Happy Prince"" and Other Stories (Penguin ..."
8,3.05444,NaN,'Salem's Lot
9,3.04748,NaN,...AND THE HORSE HE RODE IN ON : THE PEOPLE V....


In [11]:
from sklearn.decomposition import NMF, TruncatedSVD, dict_learning
import time

In [12]:
def decompose_matrix(matrix, decomposer, evaluate=False):
    """
        matrix: 2D numpy array or pandas dataframe
        decomposer: sklearn class that performs the decomposition.
                    The decomposer can be any object but it is expected
                    to implement an API similar to sklearn with a
                    `fit_transform` method and a `components_` attribute.
        evaluate: bool, whether to also evaluate the model.
    
        Decomposes a matrix with the given method and returns the two
        decomposed vectors. If evaluate=True then this also returns an
        evaluation of the decomposition.
        
        Since we don't want to treat missing values as zeros for evaluation
        but we need numbers to perform the decomposition. One way to test
        this is to add one to all the existent values, then replace NaNs
        with zeros, perform the matrix decomposition, reconstruct the matrix,
        and finally subtract one and calculate the metrics. For the resulting
        decomposition remember to subtract 1 from every future reconstruction,
        and also do min-max scaling (since recostruction may have negative or
        too-high values).        
    """
    
    matrix = (matrix+int(evaluate)).fillna(0).values

    # perform the decomposition
    User = decomposer.fit_transform(matrix)
    Item = decomposer.components_
    
    if evaluate:
        # Reconstruct the matrix and replace negatives with NaNs
        reconstructed = User @ Item - 1
        reconstructed[reconstructed<0] = np.nan
        reconstructed[reconstructed>10] = 10
    
        # R_squared, pearson correlation, and cosine similarity between 
        # the reconstructed and original matrices
        r2 = compute_evaluation_metric(matrix, reconstructed, "R2")
        rmse = compute_evaluation_metric(matrix, reconstructed, "RMSE")
        corr = similirity_evaluation(reconstructed.ravel(), matrix.ravel())[0][0]
        cosine = similirity_evaluation(reconstructed.ravel(), matrix.ravel(), "cosine")[0][0]
    
        print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))
        return User, Item
    
    return User, Item


def recreate_matrix(U, I, ):
    """
        Given User and Item decomposed vectors, it performs the necessary
        processing needed after `decompose_matrix`. It returns the full
        reconstructed matrix. It places upper and lower bounds upon ratings
        by clipping extremes rather than min/max scaling (as this seemed to
        be the best performing approach).
    """
    
    return (U @ I - 1).clip(0,10)

In [13]:
complete.shape

(12490, 13150)

In [49]:
from tqdm import tqdm

In [14]:
import tensorflow as tf

In [ ]:
sess = tf.Session()

In [164]:
n = 50

InputTensor = tf.placeholder(tf.float32, shape=complete.shape)

UserTensor = tf.Variable(tf.random_normal((complete.shape[0],n), dtype=tf.float32),
                         dtype=tf.float32)
ItemTensor = tf.Variable(tf.random_normal((n, complete.shape[1]), dtype=tf.float32),
                              dtype=tf.float32)

mul = tf.matmul(UserTensor, ItemTensor,)
diffs = tf.subtract(InputTensor, mul)
squares = tf.square(diffs)
loss = tf.reduce_sum(squares)
added_loss = (loss - tf.minimum( tf.reduce_min(UserTensor),0) 
                   - tf.minimum( tf.reduce_min(ItemTensor),0)
                   - tf.minimum( tf.reduce_min(mul),0))

opt = tf.train.AdamOptimizer(learning_rate=0.1).minimize(added_loss)
sess.run(tf.global_variables_initializer())


In [171]:
for i in tqdm(range(200)):
    sess.run(opt, feed_dict={InputTensor: complete.fillna(0).astype(np.float32).values})

100%|██████████| 200/200 [13:54<00:00,  4.17s/it]


In [172]:
U = sess.run(UserTensor)
I = sess.run(ItemTensor)

In [173]:
(U @ I).max(), (U @ I).min()

(30.601593, -6.418685)

In [174]:
reconstructed = U @ I

r2 = compute_evaluation_metric(complete, reconstructed, "R2")
rmse = compute_evaluation_metric(complete, reconstructed, "RMSE")
corr = similirity_evaluation(reconstructed.ravel(), complete.values.ravel())[0][0]
cosine = similirity_evaluation(reconstructed.ravel(), complete.values.ravel(), "cosine")[0][0]

print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))

RSq: -0.3518050265393382
RMSE: 4.422975659083189
Pearson: 0.16629667882866173
Cosine similarity: 0.24024538911729487


In [175]:
reconstructed = U @ I
reconstructed = reconstructed - reconstructed.min()


r2 = compute_evaluation_metric(complete, reconstructed, "R2")
rmse = compute_evaluation_metric(complete, reconstructed, "RMSE")
corr = similirity_evaluation(reconstructed.ravel(), complete.values.ravel())[0][0]
cosine = similirity_evaluation(reconstructed.ravel(), complete.values.ravel(), "cosine")[0][0]

print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))

RSq: -1.0647362809565801
RMSE: 5.466254091295412
Pearson: 0.16629290747995673
Cosine similarity: 0.03014516704444644


In [35]:
(U @ I).max(), (U @ I).min()

(25.966671, -26.273178)

In [98]:
from mlxtend.frequent_patterns import apriori, association_rules

In [99]:
subset = complete.fillna(0)
subset[subset<=6] = 0
subset[subset>6] = 1

In [100]:
apr = apriori(subset, min_support=0.0005, n_jobs=4) # 0.0001 is WAY too low and takes WAY too much time
apr.head(10)

,support,itemsets
0,0.000560,(3)
1,0.004243,(5)
2,0.001681,(6)
3,0.001041,(8)
4,0.000881,(12)
5,0.000801,(16)
6,0.000721,(22)
7,0.000801,(23)
8,0.006886,(28)
9,0.009448,(31)


In [101]:
apr.shape

(10483, 2)

In [135]:
apr.sort_values("support", ascending=False)

,support,itemsets
3578,0.028423,(10684)
3223,0.022818,(9741)
1396,0.018895,(4204)
3795,0.018495,(11348)
1400,0.016013,(4208)
1402,0.015853,(4212)
1823,0.015853,(5531)
3742,0.015372,(11187)
530,0.015132,(1574)
3642,0.014652,(10871)


In [137]:
ass_rules = association_rules(apr, min_threshold=0.5)
ass_rules.sort_values("confidence", ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14359,"(10582, 1634, 2022)","(1420, 4103)",0.000560,0.001681,0.000560,1.0,594.761905,0.000560,inf
21171,"(10473, 10687)","(1634, 1443, 1420, 4103)",0.000560,0.000641,0.000560,1.0,1561.250000,0.000560,inf
16222,"(3312, 4123, 4645, 2326)",(3036),0.000721,0.003363,0.000721,1.0,297.380952,0.000718,inf
18417,"(1024, 4123, 221)","(1666, 3036, 4645)",0.000560,0.000881,0.000560,1.0,1135.454545,0.000560,inf
19393,"(221, 5139, 3797)","(1666, 4123, 4978)",0.000560,0.000801,0.000560,1.0,1249.000000,0.000560,inf
6825,"(1635, 1444, 2022)",(10687),0.000721,0.002722,0.000721,1.0,367.352941,0.000719,inf
16209,"(3312, 5139, 2326)","(3036, 3797)",0.000560,0.001281,0.000560,1.0,780.625000,0.000560,inf
11004,"(1024, 221, 3797, 4645)",(3036),0.000560,0.003363,0.000560,1.0,297.380952,0.000559,inf
11006,"(1024, 221, 3036, 3797)",(4645),0.000560,0.003363,0.000560,1.0,297.380952,0.000559,inf
16207,"(5139, 3036, 3797, 2326)",(3312),0.000560,0.003122,0.000560,1.0,320.256410,0.000559,inf


In [138]:
complete.columns[[10582, 1634, 2022]]

Index(['The Laughing Corpse (Anita Blake Vampire Hunter (Paperback))',
       'Burnt Offerings (Anita Blake Vampire Hunter (Paperback))',
       'Circus of the Damned (Anita Blake Vampire Hunter (Paperback))'],
      dtype='object', name='Book-Title')

In [139]:
complete.columns[[1420, 4103]]

Index(['Bloody Bones (Anita Blake Vampire Hunter (Paperback))', 'Guilty Pleasures (Anita Blake Vampire Hunter (Paperback))'], dtype='object', name='Book-Title')

In [103]:
ass_rules.shape

(25808, 9)

In [141]:
from sklearn.neighbors import NearestNeighbors

In [142]:
knn = NearestNeighbors()

In [143]:
knn.fit(complete.fillna(0))

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [144]:
distances, idx = knn.kneighbors(complete[1:2].fillna(0))

In [145]:
idx

array([[   1,  866, 8407, 9544, 6893]])

In [146]:
complete.iloc[1,idx.ravel()]

Book-Title
 Q-Space (Star Trek The Next Generation, Book 47)         NaN
Anne of Green Gables: Three Volumes in One                NaN
Sixth Grade Can Really Kill You                           NaN
The Cat Who Played Post Office (Cat Who... (Paperback))   NaN
One Child                                                 NaN
Name: User_100009, dtype: float64

In [147]:
sims = distances.ravel()
orig = complete.iloc[1,idx.ravel()].values
results = pd.DataFrame([complete.columns[idx].ravel(), sims, orig],
                           index=["Items", "Predicted Ratings", "Original ratings"]).T

results.sort_values("Original ratings", ascending=False).reset_index(drop=True)

,Items,Predicted Ratings,Original ratings
0,"Q-Space (Star Trek The Next Generation, Book 47)",0,NaN
1,Anne of Green Gables: Three Volumes in One,29.9333,NaN
2,Sixth Grade Can Really Kill You,29.9333,NaN
3,The Cat Who Played Post Office (Cat Who... (Pa...,29.9333,NaN
4,One Child,30.0666,NaN


In [185]:
for user in complete.index[:5]:
    u = get_user(user, complete)
    user_itempreds = predict_user_ratings(u, complete)
    sims = similirity_evaluation(u, complete)
    
    dummy_scorer = compute_evaluation_metric(u, np.ones(u.shape)*np.nanmean(u))
    model_scorer = compute_evaluation_metric(u, user_itempreds)
    
    print(f"{user}:\n\tDummy: {dummy_scorer}\n\tModel: {model_scorer}\n")

User_100004:
	Dummy: 23.140495867768596
	Model: 18.278362220126084

User_100009:
	Dummy: 16.53333333333333
	Model: 15.091299868063857

User_10001:
	Dummy: 13.88888888888889
	Model: 7.741781488302394

User_100010:
	Dummy: 0.0
	Model: 6.521531911340406e-06

User_100038:
	Dummy: 0.0
	Model: 3.76230950728461e-05



In [186]:
find_similar_items(j5, complete, "cosine")

,Items,Similarity scores
0,Harry Potter and the Chamber of Secrets (Book 2),1
1,Harry Potter and the Prisoner of Azkaban (Book 3),0.578975
2,Harry Potter and the Goblet of Fire (Book 4),0.569977
3,Harry Potter and the Sorcerer's Stone (Book 1),0.451425
4,Harry Potter and the Order of the Phoenix (Boo...,0.361031
5,Harry Potter and the Sorcerer's Stone (Harry P...,0.346424
6,The Fellowship of the Ring (The Lord of the Ri...,0.160708
7,The Hobbit: or There and Back Again,0.135877
8,Dr. Seuss's A B C (I Can Read It All by Myself...,0.119482
9,Ramona the Pest (Ramona Quimby (Paperback)),0.119221


In [187]:
%%timeit
find_similar_items(j5, complete, "cosine")

2.56 s ± 41.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [188]:
%%timeit
find_similar_items(j5, complete, "pearson")

4.56 s ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
find_similar_users(i1, complete, "cosine")

,Users,Similarity scores
11487,User_8,0.99996
10849,User_67544,0.398504
814,User_116866,0.311829
5755,User_219008,0.275409
8030,User_263325,0.134908
808,User_11676,0.0200369
10734,User_65322,0.0001
9782,User_46133,0.0001
4889,User_201634,0.0001
7006,User_243200,0.0001


In [134]:
%%timeit
find_similar_users(i1, complete, "cosine")

2.49 s ± 7.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [135]:
%%timeit
find_similar_users(i1, complete, "pearson")

4.41 s ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### User-based tests:

In [173]:
similirity_evaluation(i1, i3, "pearson")

array([[-0.09138566,  0.13498689]])

In [174]:
similirity_evaluation(i1, i3, "cosine")

array([[6.62261793e-10, 3.98504416e-01]])

In [74]:
%%timeit
cos_sim(i1, i3)

148 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [75]:
%%timeit
pearson_sim(i1, i3)

500 µs ± 2.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [170]:
%%timeit
similirity_evaluation(i1, i3, "cosine")

157 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [77]:
%%timeit
similirity_evaluation(i1, i3, "pearson")

509 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Item-based tests:

In [211]:
similirity_evaluation(j2, j4, "pearson")

array([[-0.06359102,  0.02868645,  0.00793028]])

In [212]:
similirity_evaluation(j2, j4, "cosine")

array([[1.16175623e-10, 8.09315160e-11, 1.24607297e-02]])

In [161]:
%%timeit
similirity_evaluation(j2, j4, "cosine")

635 µs ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [81]:
%%timeit
similirity_evaluation(j2, j4, "pearson")

1.36 ms ± 23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [90]:
a = similirity_evaluation(i1, complete)

In [91]:
a[1:][np.argmax(a[1:])]

array([0.2022545])

In [92]:
np.argmax(a[1:])

11326

In [93]:
complete.iloc[[0, 11326+1]].dropna(thresh=1, axis=1)

ISBN,0002005018,0140119906,0312265840,0312306326,0316780375,0399135782,0440236738,0679770151,0767905385,0786866586,0821749528
User-ID,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
252282,NaN,9.0,6.0,3.0,0.0,0.0,0.0,7.0,0.0,5.0,0.0


In [96]:
# find rows with common ratings with i2
ff3 = complete.T[((~np.isnan(i2)) & (~np.isnan(complete))).any(0)].T
ff3

ISBN,0002005018,0060084405,0060175532,0060512822,0060958022,0140126562,0140430016,0151008116,0312195516,0312274920,...,0671021001,067166641X,0679449434,0743225325,074341134X,0743418174,0786868716,1551664992,1552781542,1853261912
User-ID,,,,,,,,,,,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
ff3.shape

(12490, 38)

In [252]:
# drop rows and columns with less than 5 common ratings
nnans3 = ff3.dropna(thresh=2, axis=1).dropna(thresh=2, axis=0)
nnans3

ISBN,0002005018,0060084405,0060175532,0060512822,0060958022,0140126562,0140430016,0151008116,0312195516,0312274920,...,0671021001,067166641X,0679449434,0743225325,074341134X,0743418174,0786868716,1551664992,1552781542,1853261912
User-ID,,,,,,,,,,,,,,,,,,,,,
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1424,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
2110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2766,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3363,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3373,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
nnans3.shape

(883, 38)

In [296]:
# Find the similarity between the new vector and the candidates
i2_short = nnans3.loc[:,"0151008116"]
similirity_evaluation(i2_short, nnans3.T, "cosine").round(2)

array([[0.11, 0.  , 0.  , 0.07, 0.11, 0.06, 0.  , 1.  , 0.18, 0.02, 0.12,
        0.07, 0.05, 0.09, 0.  , 0.  , 0.03, 0.06, 0.18, 0.18, 0.1 , 0.  ,
        0.01, 0.05, 0.05, 0.03, 0.08, 0.  , 0.08, 0.  , 0.07, 0.04, 0.12,
        0.12, 0.13, 0.13, 0.14, 0.  ]])

In [255]:
similirity_evaluation(i2_short, nnans3.loc[:,"0151008116"].T, "cosine").round(2)

array([[0.11]])

In [299]:
nnans3.shape, i2_short.shape

((883, 38), (883,))

In [256]:
# check original (only common values) to see if 0.44 correlation appears plausible 
complete.loc[:,["0060958022","0151008116"]].dropna(thresh=2, axis=0)

ISBN,0060958022,0151008116
User-ID,,
11400,10.0,6.0
11676,3.0,6.0
43246,0.0,0.0
98741,0.0,0.0
113519,8.0,9.0
114868,0.0,0.0
139742,0.0,0.0
172742,0.0,0.0
201290,0.0,0.0


In [275]:
ii1 = complete.dropna(thresh=2, axis=0).T.values
pearsonr(ii1[0,:], ii1[1,:])

(nan, 1.0)

In [277]:
ii1.shape

(14755, 11931)

In [326]:
predict_i(i1, complete.values)

array([5.10557647, 2.5       , 2.3248965 , ..., 2.5       , 2.5       ,
       2.5       ])

In [327]:
predict_i(i2, complete.values)

array([3.00031253, 3.06117812, 3.04107743, ..., 3.05825516, 3.06124779,
       3.05160399])

In [328]:
predict_i(i4, complete)

array([6.76274148, 6.625     , 6.61740402, ..., 6.625     , 6.64395034,
       6.63900064])

In [329]:
pj1 = predict_i(get_item("0060958022"), complete.T.values)
pj1

array([3.28665504, 3.28701045, 3.28753367, ..., 3.28563866, 3.28397332,
       3.28902673])

In [330]:
pj1.max(), pj1.min()

(5.749083253619088, 3.0239473786652513)

In [331]:
pj2 = predict_i(get_item("0151008116"), complete.T.values)
pj2

array([4.17182472, 4.17182336, 4.17366063, ..., 4.16760712, 4.17522016,
       4.1735431 ])

In [332]:
pj2.max(), pj2.min()

(6.678546628117268, 3.9005324537219246)

In [333]:
pj2 = predict_i(get_item("0151008116"), complete.T.values)
pj2

array([4.17182472, 4.17182336, 4.17366063, ..., 4.16760712, 4.17522016,
       4.1735431 ])

In [334]:
preds = predict_i(i2_short, nnans3.T)
preds

array([3.25771254, 3.15944676, 3.28837306, 3.60032211, 3.69521197,
       3.59819321, 3.45268945, 3.01250799, 3.4767577 , 3.07625759,
       3.07192041, 3.65018902, 3.03111267, 2.30878096, 3.81809197,
       3.1903585 , 3.16149781, 4.00726237, 2.18888988, 3.20429946,
       3.24694719, 3.71515639, 3.93581135, 3.55492262, 3.23461876,
       2.36959207, 3.39089289, 3.06199426, 3.17407433, 3.70178995,
       3.2847156 , 3.80698116, 5.03280303, 3.28568041, 6.26741684,
       3.2514888 , 3.59133727, 3.63270423, 2.22733659, 3.73486387,
       3.62608575, 3.40048946, 3.14417427, 2.88790841, 3.28241253,
       3.5056749 , 3.15691557, 3.04741394, 3.83755428, 7.42219818,
       3.32384496, 3.30654764, 3.43129087, 3.1835838 , 3.74798546,
       3.17888678, 3.33788524, 3.16149781, 3.42508768, 3.53569926,
       3.64214881, 2.55892388, 3.49219069, 3.1907591 , 3.31094932,
       3.78381863, 3.2478728 , 3.72279069, 3.63115602, 2.74138293,
       3.3205056 , 3.47809287, 3.59898504, 3.7230413 , 3.52081

In [335]:
preds.max(), preds.mean(), preds.min(), preds.std()

(7.422198182599594, 3.3698630136986303, 1.6851247135632899, 0.4790006614074269)

In [336]:
preds.argmax() # who is the "closest user" (in essense, who is the user most likely to rate this well)

49

In [337]:
nnans3.iloc[49]

ISBN
0002005018     NaN
0060084405     NaN
0060175532     NaN
0060512822     NaN
0060958022     NaN
0140126562     NaN
0140430016     NaN
0151008116    10.0
0312195516     9.0
0312274920     NaN
0312305060     8.0
0312978383     6.0
0316789089     NaN
0345445848     0.0
0345445856     NaN
0375504036     NaN
0375506039     NaN
0375724850     NaN
0375727345     9.0
0380723085    10.0
0385333412     NaN
0425116840     0.0
0440211727     0.0
0440222656     9.0
0446364193     8.0
0446606324     0.0
0449003981     NaN
0553582658     0.0
0671021001    10.0
067166641X     NaN
0679449434     NaN
0743225325     NaN
074341134X    10.0
0743418174     9.0
0786868716     8.0
1551664992     NaN
1552781542     NaN
1853261912     NaN
Name: 16795, dtype: float64

In [338]:
nnans3.iloc[49]["0151008116"] # seems legit

10.0

In [339]:
similirity_evaluation(get_item("0151008116"), get_item("0380723085"), "cosine")

array([[0.11914448]])